In [8]:
import os
import random
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns 
import matplotlib.cm as cm
from matplotlib import rcParams
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.isri import ISRIStemmer
from collections import Counter 
import itertools
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from joblib import dump, load
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\luche\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# data import 

In [26]:
import pandas as pd

df = pd.read_pickle('./data_exclamation.pkl')
#dataTest = pd.read_table('../input/semevalll/SemEval2017-test.txt', usecols=[1,2], encoding='utf-8', names=['sentiment', 'tweet'])                   
#combine = [dataTrain,dataTest]
df

,id,sentiment,tweet,label,embedding,tokenized,number of exclamations
0,628949369883000832,negative,dear the newOoffice for Mac is great and all ...,-1,"[118309.0, 372306.0, 229153.0, 166369.0, 22830...","[dear, newooffice, mac, great, lync, update, c...",0
1,628976607420645377,negative,how about you make a system that doesnt eat m...,-1,"[231458.0, 350362.0, 126852.0, 132701.0, 15409...","[make, system, doesnt, eat, friggin, disc, tim...",1
2,629023169169518592,negative,I may be ignorant on this issue but should we ...,-1,"[237383.0, 186885.0, 193637.0, 94875.0, 277466...","[may, ignorant, issue, celebrate, parental, le...",0
3,629179223232479232,negative,Thanks to I just may be switching over to,-1,"[357161.0, 237383.0, 349661.0, 0.0, 0.0, 0.0, ...","[thanks, may, switch]",0
4,629186282179153920,neutral,If I make a game as a windows Universal App Wi...,0,"[231458.0, 157049.0, 388091.0, 372227.0, 57667...","[make, game, window, universal, app, xboxone, ...",0
...,...,...,...,...,...,...,...
5863,639855845958885376,positive,ok good to know Punting at MetLife in Decembe...,1,"[164328.0, 209645.0, 295361.0, 242311.0, 11868...","[good, know, punt, metlife, december, task, ho...",0
5864,639979760735662080,neutral,everyone who sat around me at metlife was so a...,0,"[141948.0, 333081.0, 59937.0, 242311.0, 55915....","[everyone, sit, around, metlife, annoying, did...",0
5865,640196838260363269,neutral,what giants or niners fans would wanna go to t...,0,"[161068.0, 261597.0, 144744.0, 390139.0, 38289...","[giants, niner, fan, would, wan, sunday, night...",0
5866,640975710354567168,positive,Anybody want a ticket for tomorrow Colombia vs...,1,"[57178.0, 383068.0, 359356.0, 361707.0, 105227...","[anybody, want, ticket, tomorrow, colombia, pe...",0


In [50]:
print(Counter(df['label']))

Counter({1: 3017, 0: 2001, -1: 850})


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5868 entries, 0 to 5867
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      5868 non-null   int64 
 1   sentiment               5868 non-null   object
 2   tweet                   5868 non-null   object
 3   label                   5868 non-null   int64 
 4   embedding               5868 non-null   object
 5   tokenized               5868 non-null   object
 6   number of exclamations  5868 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 366.8+ KB


In [30]:
X.dtype

dtype('float64')

# add label

In [4]:
CATEGORY_INDEX = {
    "negative": -1,
    "neutral": 0,
    "positive": 1
}

"""import data """
raw_label = df['sentiment'].values.tolist()
rawlabel = []
for i in range(len(raw_label)):
    rawlabel.append(CATEGORY_INDEX[raw_label[i]])
rawlabel

[-1,
 -1,
 -1,
 -1,
 0,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 0,
 1,
 0,
 0,
 -1,
 -1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 1,
 0,
 -1,
 -1,
 -1,
 0,
 0,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 0,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 0,
 1,
 0,
 -1,
 -1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 -1,
 1,
 0,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 1,
 0,
 1,
 -1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 -1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 -1,
 1,
 1,
 1,
 1,
 -1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 -1,
 0,
 -1,
 1,
 -1,
 1,
 1,
 1,
 -1,
 1,
 -1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 -1,
 0

In [5]:
df['label'] = rawlabel
df

,id,sentiment,tweet,label
0,628949369883000832,negative,dear @Microsoft the newOoffice for Mac is grea...,-1
1,628976607420645377,negative,@Microsoft how about you make a system that do...,-1
2,629023169169518592,negative,I may be ignorant on this issue but... should ...,-1
3,629179223232479232,negative,"Thanks to @microsoft, I just may be switching ...",-1
4,629186282179153920,neutral,If I make a game as a #windows10 Universal App...,0
...,...,...,...,...
5863,639855845958885376,positive,@Racalto_SK ok good to know. Punting at MetLif...,1
5864,639979760735662080,neutral,everyone who sat around me at metlife was so a...,0
5865,640196838260363269,neutral,what giants or niners fans would wanna go to t...,0
5866,640975710354567168,positive,Anybody want a ticket for tomorrow Colombia vs...,1


In [6]:
df_back = df.copy()
df_back

,id,sentiment,tweet,label
0,628949369883000832,negative,dear @Microsoft the newOoffice for Mac is grea...,-1
1,628976607420645377,negative,@Microsoft how about you make a system that do...,-1
2,629023169169518592,negative,I may be ignorant on this issue but... should ...,-1
3,629179223232479232,negative,"Thanks to @microsoft, I just may be switching ...",-1
4,629186282179153920,neutral,If I make a game as a #windows10 Universal App...,0
...,...,...,...,...
5863,639855845958885376,positive,@Racalto_SK ok good to know. Punting at MetLif...,1
5864,639979760735662080,neutral,everyone who sat around me at metlife was so a...,0
5865,640196838260363269,neutral,what giants or niners fans would wanna go to t...,0
5866,640975710354567168,positive,Anybody want a ticket for tomorrow Colombia vs...,1


# data preprocessing

## 1 data cleansing 

In [7]:
df = df.dropna()

In [8]:
def clean_base(tweets, clean_object):
        #tweets.loc[:, "tweet"].replace(clean_object, "", inplace=True)
        tweets = re.sub(clean_object, ' ', tweets)
        return tweets

In [9]:
def remove_urls(tweets):
        return clean_base(tweets, re.compile(r"http.?://[^\s]+[\s]?"))

In [10]:
def remove_usernames(tweets):
        return clean_base(tweets, re.compile(r"@[^\s]+[\s]?"))

In [11]:
def remove_hashtags(tweets):  # it unrolls the hashtags to normal words
        for hashtag in map(lambda x: re.compile(re.escape(x)), [",", ":", "\"", "=", "&", ";", "%", "$",
                                                                     "@", "%", "^", "*", "(", ")", "{", "}",
                                                                     "[", "]", "|", "/", "\\", ">", "<", "-",
                                                                     "!", "?", ".", "'",
                                                                     "--", "---", "#"]):
            tweets = re.sub(hashtag, ' ', tweets)
        return tweets

In [12]:
def remove_numbers(tweets):
        return clean_base(tweets, re.compile(r"\s?[0-9]+\.?[0-9]*"))

In [13]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [14]:
##############################################

In [15]:
english_punctuations = string.punctuation
punctuations_list = english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [16]:
###############################################################

In [17]:
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = remove_usernames(doc)
    #Replace url with empty string
    doc = remove_urls(doc)

    
    #doc = re.sub(r'@[^\s]+', ' ', doc)
    #doc = re.sub(r'_', ' ', doc)
    
    
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('(www\.[^\s])',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    #doc= normalize_arabic(doc)
    
    #Replace numbers with empty string
    doc = remove_numbers(doc)
    #Replace @username with empty string
    doc = remove_hashtags(doc)
    # remove repeated letters
    #doc=remove_repeating_char(doc)
    
    #stemming
    doc = stemmer.stem(doc)
   
    return doc



stemmer = ISRIStemmer()
df["tweet"] = df['tweet'].apply(lambda x: processDocument(x, stemmer))
df

,id,sentiment,tweet,label
0,628949369883000832,negative,dear the newOoffice for Mac is great and all ...,-1
1,628976607420645377,negative,how about you make a system that doesnt eat m...,-1
2,629023169169518592,negative,I may be ignorant on this issue but should we ...,-1
3,629179223232479232,negative,Thanks to I just may be switching over to,-1
4,629186282179153920,neutral,If I make a game as a windows Universal App Wi...,0
...,...,...,...,...
5863,639855845958885376,positive,ok good to know Punting at MetLife in Decembe...,1
5864,639979760735662080,neutral,everyone who sat around me at metlife was so a...,0
5865,640196838260363269,neutral,what giants or niners fans would wanna go to t...,0
5866,640975710354567168,positive,Anybody want a ticket for tomorrow Colombia vs...,1


In [98]:
#df.to_csv('./data.csv')

# tokenization and stem

In [18]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luche\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\luche\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [20]:
########################################trail 1#######################################################

In [21]:
data = []

# Separating our features (text) and our labels into two lists to smoothen our work
X = df['tweet'].tolist()
Y = df['label'].tolist()

# Building our data list, that is a list of tuples, where each tuple is a pair of the tokenized text
# and its corresponding label
for x, y in zip(X, Y):
    
    data.append((nltk.word_tokenize(x), y))
    
# Printing the CPU time and the first 5 elements of our 'data' list
#print('CPU Time:', time() - start_time)
data[:5]

[(['dear',
   'the',
   'newOoffice',
   'for',
   'Mac',
   'is',
   'great',
   'and',
   'all',
   'but',
   'no',
   'Lync',
   'update',
   'Cmon'],
  -1),
 (['how',
   'about',
   'you',
   'make',
   'a',
   'system',
   'that',
   'doesnt',
   'eat',
   'my',
   'friggin',
   'discs',
   'This',
   'is',
   'the',
   'nd',
   'time',
   'this',
   'has',
   'happened',
   'and',
   'I',
   'am',
   'so',
   'sick',
   'of',
   'it'],
  -1),
 (['I',
   'may',
   'be',
   'ignorant',
   'on',
   'this',
   'issue',
   'but',
   'should',
   'we',
   'celebrate',
   'parental',
   'leave',
   'changes',
   'Doesnt',
   'the',
   'gender',
   'divide',
   'suggest'],
  -1),
 (['Thanks', 'to', 'I', 'just', 'may', 'be', 'switching', 'over', 'to'], -1),
 (['If',
   'I',
   'make',
   'a',
   'game',
   'as',
   'a',
   'windows',
   'Universal',
   'App',
   'Will',
   'xboxone',
   'owners',
   'be',
   'able',
   'to',
   'download',
   'and',
   'play',
   'it',
   'in',
   'Novemb

In [7]:
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from time import time

In [36]:
pos_tag(data[0][0])

[('dear', 'IN'),
 ('the', 'DT'),
 ('newOoffice', 'NN'),
 ('for', 'IN'),
 ('Mac', 'NNP'),
 ('is', 'VBZ'),
 ('great', 'JJ'),
 ('and', 'CC'),
 ('all', 'DT'),
 ('but', 'CC'),
 ('no', 'DT'),
 ('Lync', 'NNP'),
 ('update', 'NN'),
 ('Cmon', 'NNP')]

In [23]:
STOP_WORDS = stopwords.words('english')
def lemmatize_sentence(tweet_tokens):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        # Eliminating the token if it is a link
        
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token.lower(), pos)

        
        # Eliminating the token if its length is less than 3, if it is a punctuation or if it is a stopword
        if token not in string.punctuation and len(token) > 2 and token not in STOP_WORDS:
            cleaned_tokens.append(token)
            
    return cleaned_tokens
# Prevewing the remove_noise() output
print(lemmatize_sentence(data[0][0]))

['dear', 'newooffice', 'mac', 'great', 'lync', 'update', 'cmon']


In [28]:
start_time = time()

# As the Naive Bayesian classifier accepts inputs in a dict-like structure,
# we have to define a function that transforms our data into the required input structure
def list_to_dict(cleaned_tokens):
    return dict([token, True] for token in cleaned_tokens)

cleaned_tokens_list = []

# Removing noise from all the data
for tokens, label in data:
    cleaned_tokens_list.append((lemmatize_sentence(tokens), label))

print('Removed Noise, CPU Time:', time() - start_time)
start_time = time()

final_data = []

# Transforming the data to fit the input structure of the Naive Bayesian classifier
for tokens, label in cleaned_tokens_list:
    final_data.append((list_to_dict(tokens), label))
    
print('Data Prepared for model, CPU Time:', time() - start_time)

# Previewing our final (tokenized, cleaned and lemmatized) data list
final_data[:5]

Removed Noise, CPU Time: 35.368741512298584
Data Prepared for model, CPU Time: 0.02720952033996582


[({'dear': True,
   'newooffice': True,
   'mac': True,
   'great': True,
   'lync': True,
   'update': True,
   'cmon': True},
  -1),
 ({'make': True,
   'system': True,
   'doesnt': True,
   'eat': True,
   'friggin': True,
   'disc': True,
   'time': True,
   'happen': True,
   'sick': True},
  -1),
 ({'may': True,
   'ignorant': True,
   'issue': True,
   'celebrate': True,
   'parental': True,
   'leave': True,
   'change': True,
   'doesnt': True,
   'gender': True,
   'divide': True,
   'suggest': True},
  -1),
 ({'thanks': True, 'may': True, 'switch': True}, -1),
 ({'make': True,
   'game': True,
   'window': True,
   'universal': True,
   'app': True,
   'xboxone': True,
   'owner': True,
   'able': True,
   'download': True,
   'play': True,
   'november': True},
  0)]

## save tokenized data

In [110]:
df['tokenized'] = ''
for i in range(len(df)):
    df['tokenized'][i] = cleaned_tokens_list[i][0]
df

C:\Users\luche\AppData\Local\Temp/ipykernel_2568/1798639570.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokenized'][i] = cleaned_tokens_list[i][0]


,id,sentiment,tweet,label,embedding,tokenized
0,628949369883000832,negative,dear the newOoffice for Mac is great and all ...,-1,"[118309.0, 372306.0, 229153.0, 166369.0, 22830...","[dear, newooffice, mac, great, lync, update, c..."
1,628976607420645377,negative,how about you make a system that doesnt eat m...,-1,"[231458.0, 350362.0, 126852.0, 132701.0, 15409...","[make, system, doesnt, eat, friggin, disc, tim..."
2,629023169169518592,negative,I may be ignorant on this issue but should we ...,-1,"[237383.0, 186885.0, 193637.0, 94875.0, 277466...","[may, ignorant, issue, celebrate, parental, le..."
3,629179223232479232,negative,Thanks to I just may be switching over to,-1,"[357161.0, 237383.0, 349661.0, 0.0, 0.0, 0.0, ...","[thanks, may, switch]"
4,629186282179153920,neutral,If I make a game as a windows Universal App Wi...,0,"[231458.0, 157049.0, 388091.0, 372227.0, 57667...","[make, game, window, universal, app, xboxone, ..."
...,...,...,...,...,...,...
5863,639855845958885376,positive,ok good to know Punting at MetLife in Decembe...,1,"[164328.0, 209645.0, 295361.0, 242311.0, 11868...","[good, know, punt, metlife, december, task, ho..."
5864,639979760735662080,neutral,everyone who sat around me at metlife was so a...,0,"[141948.0, 333081.0, 59937.0, 242311.0, 55915....","[everyone, sit, around, metlife, annoying, did..."
5865,640196838260363269,neutral,what giants or niners fans would wanna go to t...,0,"[161068.0, 261597.0, 144744.0, 390139.0, 38289...","[giants, niner, fan, would, wan, sunday, night..."
5866,640975710354567168,positive,Anybody want a ticket for tomorrow Colombia vs...,1,"[57178.0, 383068.0, 359356.0, 361707.0, 105227...","[anybody, want, ticket, tomorrow, colombia, pe..."


In [29]:
#import pickle
#pickle.dump(cleaned_tokens_list, open("./data.p" , "wb"))

## extra feature 

In [114]:
df_back['tweet']

0       dear @Microsoft the newOoffice for Mac is grea...
1       @Microsoft how about you make a system that do...
2       I may be ignorant on this issue but... should ...
3       Thanks to @microsoft, I just may be switching ...
4       If I make a game as a #windows10 Universal App...
                              ...                        
5863    @Racalto_SK ok good to know. Punting at MetLif...
5864    everyone who sat around me at metlife was so a...
5865    what giants or niners fans would wanna go to t...
5866    Anybody want a ticket for tomorrow Colombia vs...
5867    Mendez told me he'd drive me to MetLife on Sun...
Name: tweet, Length: 5868, dtype: object

In [112]:
def count_occurences(character, word_array):
            counter = 0
            for j, word in enumerate(word_array):
                for char in word:
                    if char == character:
                        counter += 1
            return counter
    

In [115]:
exclamations = list(map(lambda txt: count_occurences("!", txt),
                                df_back['tweet']))



In [118]:
df['number of exclamations'] = exclamations

,id,sentiment,tweet,label,embedding,tokenized,number of exclamations
0,628949369883000832,negative,dear the newOoffice for Mac is great and all ...,-1,"[118309.0, 372306.0, 229153.0, 166369.0, 22830...","[dear, newooffice, mac, great, lync, update, c...",0
1,628976607420645377,negative,how about you make a system that doesnt eat m...,-1,"[231458.0, 350362.0, 126852.0, 132701.0, 15409...","[make, system, doesnt, eat, friggin, disc, tim...",1
2,629023169169518592,negative,I may be ignorant on this issue but should we ...,-1,"[237383.0, 186885.0, 193637.0, 94875.0, 277466...","[may, ignorant, issue, celebrate, parental, le...",0
3,629179223232479232,negative,Thanks to I just may be switching over to,-1,"[357161.0, 237383.0, 349661.0, 0.0, 0.0, 0.0, ...","[thanks, may, switch]",0
4,629186282179153920,neutral,If I make a game as a windows Universal App Wi...,0,"[231458.0, 157049.0, 388091.0, 372227.0, 57667...","[make, game, window, universal, app, xboxone, ...",0
...,...,...,...,...,...,...,...
5863,639855845958885376,positive,ok good to know Punting at MetLife in Decembe...,1,"[164328.0, 209645.0, 295361.0, 242311.0, 11868...","[good, know, punt, metlife, december, task, ho...",0
5864,639979760735662080,neutral,everyone who sat around me at metlife was so a...,0,"[141948.0, 333081.0, 59937.0, 242311.0, 55915....","[everyone, sit, around, metlife, annoying, did...",0
5865,640196838260363269,neutral,what giants or niners fans would wanna go to t...,0,"[161068.0, 261597.0, 144744.0, 390139.0, 38289...","[giants, niner, fan, would, wan, sunday, night...",0
5866,640975710354567168,positive,Anybody want a ticket for tomorrow Colombia vs...,1,"[57178.0, 383068.0, 359356.0, 361707.0, 105227...","[anybody, want, ticket, tomorrow, colombia, pe...",0


## load saved data

In [9]:
data_root = './data.p'

def load_data(root):
    """Load data from saved files generated in 'dataset_preprocess.py' """
    data = pickle.load(open(root, "rb"))
    return data

cleaned_tokens_list = load_data(data_root)

In [10]:
import numpy as np

In [11]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r', encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [12]:
# Loading the 50-dimensional GloVe embeddings
# This method will return three dictionaries:
# * word_to_index: a dictionary mapping from words to their indices in the vocabulary
# * index_to_word: dictionary mapping from indices to their corresponding words in the vocabulary
# * word_to_vec_map: dictionary mapping words to their GloVe vector representation
# Note that there are 400,001 words, with the valid indices ranging from 0 to 400,000
glove_file = 'glove.6B.100d.txt'
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs(glove_file)


In [13]:
def cosine_similarity(u, v):
    dot = np.dot(u, v)
    norm_u = np.sqrt(np.sum(u**2))
    norm_v = np.sqrt(np.sum(v**2))
    cosine_similarity = dot / (norm_u * norm_v)
    return cosine_similarity

In [14]:
start_time = time()

unks = []
UNKS = []

# This function will act as a "last resort" in order to try and find the word
# in the words embedding layer. It will basically eliminate contiguously occuring
# instances of a similar character
def cleared(word):
    res = ""
    prev = None
    for char in word:
        if char == prev: continue
        prev = char
        res += char
    return res


def sentence_to_indices(sentence_words, word_to_index, max_len, i):
    global X, Y
    sentence_indices = []
    for j, w in enumerate(sentence_words):
        try:
            index = word_to_index[w]
        except:
            UNKS.append(w)
            w = cleared(w)
            try:
                index = word_to_index[w]
            except:
                index = word_to_index['unk']
                unks.append(w)
        X[i, j] = index


# Here we will utilize the already computed 'cleaned_tokens_list' variable
   
print('Removed Noise, CPU Time:', time() - start_time)
start_time = time()

list_len = [len(i) for i, j in cleaned_tokens_list]
max_len = max(list_len)
print('max_len:', max_len)

X = np.zeros((len(cleaned_tokens_list), max_len))
Y = np.zeros((len(cleaned_tokens_list), ))

for i, tk_lb in enumerate(cleaned_tokens_list):
    tokens, label = tk_lb
    sentence_to_indices(tokens, word_to_index, max_len, i)
    Y[i] = label
    
print('Data Prepared for model, CPU Time:', time() - start_time)


print(X[:100])
print(Y[:100])

Removed Noise, CPU Time: 0.0009965896606445312
max_len: 536
Data Prepared for model, CPU Time: 0.04639863967895508
[[118309. 372306. 229153. ...      0.      0.      0.]
 [231458. 350362. 126852. ...      0.      0.      0.]
 [237383. 186885. 193637. ...      0.      0.      0.]
 ...
 [227517. 169725. 174666. ...      0.      0.      0.]
 [278611. 366910. 188929. ...      0.      0.      0.]
 [237383. 329448. 248238. ...      0.      0.      0.]]
[-1. -1. -1. -1.  0.  1. -1. -1.  1. -1.  1. -1. -1.  0.  1.  0.  0. -1.
 -1.  1.  1. -1.  1. -1.  1.  1.  0. -1. -1. -1.  0.  0. -1. -1. -1. -1.
 -1.  1.  1.  1.  1. -1. -1.  1.  1. -1. -1. -1.  0. -1.  1.  1.  1. -1.
 -1. -1.  1.  1.  1.  1.  1. -1. -1. -1. -1.  0.  1.  0. -1. -1.  1.  0.
  0.  1.  1.  1.  0.  1. -1.  1.  0. -1.  1. -1.  1. -1.  1. -1.  1. -1.
 -1.  1. -1.  1.  1.  0.  1. -1.  1.  1.]


In [102]:
X[:5]

array([[118309., 372306., 229153., ...,      0.,      0.,      0.],
       [231458., 350362., 126852., ...,      0.,      0.,      0.],
       [237383., 186885., 193637., ...,      0.,      0.,      0.],
       [357161., 237383., 349661., ...,      0.,      0.,      0.],
       [231458., 157049., 388091., ...,      0.,      0.,      0.]])

In [45]:
X

array([[118309., 372306., 229153., ...,      0.,      0.,      0.],
       [231458., 350362., 126852., ...,      0.,      0.,      0.],
       [237383., 186885., 193637., ...,      0.,      0.,      0.],
       ...,
       [161068., 261597., 144744., ...,      0.,      0.,      0.],
       [ 57178., 383068., 359356., ...,      0.,      0.,      0.],
       [240866., 355345., 175719., ...,      0.,      0.,      0.]])

In [105]:
df

,id,sentiment,tweet,label
0,628949369883000832,negative,dear the newOoffice for Mac is great and all ...,-1
1,628976607420645377,negative,how about you make a system that doesnt eat m...,-1
2,629023169169518592,negative,I may be ignorant on this issue but should we ...,-1
3,629179223232479232,negative,Thanks to I just may be switching over to,-1
4,629186282179153920,neutral,If I make a game as a windows Universal App Wi...,0
...,...,...,...,...
5863,639855845958885376,positive,ok good to know Punting at MetLife in Decembe...,1
5864,639979760735662080,neutral,everyone who sat around me at metlife was so a...,0
5865,640196838260363269,neutral,what giants or niners fans would wanna go to t...,0
5866,640975710354567168,positive,Anybody want a ticket for tomorrow Colombia vs...,1


In [106]:
df['embedding'] = ''
for i in range(len(df)):
    df['embedding'][i] = X[i]
df

C:\Users\luche\AppData\Local\Temp/ipykernel_2568/186184679.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['embedding'][i] = X[i]


,id,sentiment,tweet,label,embedding
0,628949369883000832,negative,dear the newOoffice for Mac is great and all ...,-1,"[118309.0, 372306.0, 229153.0, 166369.0, 22830..."
1,628976607420645377,negative,how about you make a system that doesnt eat m...,-1,"[231458.0, 350362.0, 126852.0, 132701.0, 15409..."
2,629023169169518592,negative,I may be ignorant on this issue but should we ...,-1,"[237383.0, 186885.0, 193637.0, 94875.0, 277466..."
3,629179223232479232,negative,Thanks to I just may be switching over to,-1,"[357161.0, 237383.0, 349661.0, 0.0, 0.0, 0.0, ..."
4,629186282179153920,neutral,If I make a game as a windows Universal App Wi...,0,"[231458.0, 157049.0, 388091.0, 372227.0, 57667..."
...,...,...,...,...,...
5863,639855845958885376,positive,ok good to know Punting at MetLife in Decembe...,1,"[164328.0, 209645.0, 295361.0, 242311.0, 11868..."
5864,639979760735662080,neutral,everyone who sat around me at metlife was so a...,0,"[141948.0, 333081.0, 59937.0, 242311.0, 55915...."
5865,640196838260363269,neutral,what giants or niners fans would wanna go to t...,0,"[161068.0, 261597.0, 144744.0, 390139.0, 38289..."
5866,640975710354567168,positive,Anybody want a ticket for tomorrow Colombia vs...,1,"[57178.0, 383068.0, 359356.0, 361707.0, 105227..."


In [16]:
from keras.preprocessing.sequence import pad_sequences

In [17]:
import keras
from keras import Sequential
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Bidirectional,GlobalMaxPool1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [18]:
my_seed = 22
np.random.seed(my_seed)

In [15]:
from tensorflow.keras.utils import to_categorical
Y1 = to_categorical(Y,3)
Y1

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)

In [19]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index, max_len):
    vocab_len = len(word_to_index) + 1
    emb_dim = word_to_vec_map["unk"].shape[0] #50
    
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]
        
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False, input_shape=(max_len,))
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [20]:
# Defining a sequencial model composed of firstly the embedding layer, than a pair of Bidirectional LSTMs,
# that finally feed into a sigmoid layer that generates our desired output betwene 0 and 1.

model = Sequential()

model.add(pretrained_embedding_layer(word_to_vec_map, word_to_index, max_len))
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))

model.add(Dropout(rate=0.4))
model.add(Bidirectional(LSTM(units=128, return_sequences=False)))

model.add(Dropout(rate=0.4))
#model.add(Dense(units=1, activation='sigmoid'))

model.add(Dense(units=64,kernel_initializer="uniform",activation='relu'))        
model.add(Dense(units=3,kernel_initializer="uniform",activation='softmax'))


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 536, 100)          40000100  
_________________________________________________________________
bidirectional (Bidirectional (None, 536, 128)          84480     
_________________________________________________________________
dropout (Dropout)            (None, 536, 128)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1

In [21]:
# Defining a sequencial model composed of firstly the embedding layer, than a pair of Bidirectional LSTMs,
# that finally feed into a sigmoid layer that generates our desired output betwene 0 and 1.

model = Sequential()

model.add(pretrained_embedding_layer(word_to_vec_map, word_to_index, max_len))
model.add(Bidirectional(LSTM(256,return_sequences=True), merge_mode = 'concat'))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense((3), activation="softmax"))


model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 536, 100)          40000100  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 536, 512)          731136    
_________________________________________________________________
global_max_pooling1d (Global (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                

In [34]:
trainx = df.filter(['embedding','number of exclamations']).values


AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [49]:
df['number of exclamations'].values

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [22]:
optimizer =tf.optimizers.Adam(learning_rate=0.001)
model.compile(loss='CategoricalCrossentropy', optimizer=optimizer, metrics=['accuracy'])

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(trainx, Y1, test_size=0.2, random_state=0, stratify=Y1)

In [92]:
filepath='./data/weights.best.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath= filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
earlyStop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=20, mode='min', verbose=1, restore_best_weights = True)
callbacks_list = [checkpoint]

In [25]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = 50, batch_size = 128, shuffle=True)
#model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = 50, batch_size = 128, shuffle=True,callbacks=callbacks_list)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
model.save('./lstm-4-7.h5')

In [ ]:
def plot_acc_loss(history):

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)

    plt.plot(epochs, acc, 'bo', label = 'Training Accuracy')
    plt.plot(epochs, val_acc, 'r', label = 'Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.figure()
    plt.plot(epochs, loss, 'bo', label = 'Training Loss')
    plt.plot(epochs, val_loss, 'r', label = 'Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
plot_acc_loss(model.history)